https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/08-deep-learning/notebook.ipynb

The model was trained as described in this kaggle notebook: https://www.kaggle.com/bnzn261029/capstone1-ham10k-skincancer

***Disclaimer: This work is part of an educational project. It is not intended for clinical application. As such it can not make real world predictions for skin lesions. To get recommendations regarding skin lesions one should ask for expert advice such as provided by a dermatologist.***

In [1]:
! pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 182 kB 27.1 MB/s 


In [2]:
from huggingface_hub import login
login()

Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


In [3]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 20.3 MB/s 
     |████████████████████████████████| 7.6 MB 50.4 MB/s 


In [9]:
from transformers import ImageClassificationPipeline
from transformers import TFAutoModel
from transformers import TFPreTrainedModel

In [17]:
import huggingface_hub

In [27]:
from huggingface_hub import Repository
repo = Repository(local_dir="repo", clone_from="bsenst/skin-cancer-HAM10k")

Cloning https://huggingface.co/bsenst/skin-cancer-HAM10k into local empty directory.


Download file xception_v4_1_07_0.699.h5:   0%|          | 3.48k/80.0M [00:00<?, ?B/s]

Clean file xception_v4_1_07_0.699.h5:   0%|          | 1.00k/80.0M [00:00<?, ?B/s]

In [28]:
import keras
model = keras.models.load_model("/content/repo/xception_v4_1_07_0.699.h5")

In [40]:
from tensorflow.keras.preprocessing.image import load_img
# ! wget "https://upload.wikimedia.org/wikipedia/commons/thumb/6/6c/Melanoma.jpg/320px-Melanoma.jpg"
img = load_img("320px-Melanoma.jpg", target_size=(150, 150))
import numpy as np
x = np.array(img)
X = np.array([x])
X.shape
from tensorflow.keras.applications.xception import preprocess_input
X = preprocess_input(X)
pred = model.predict(X)
classes = [
    "akiec",
    "bcc",
    "bkl",
    "df",
    "mel",
    "nv",
    "vasc",
]
dict(zip(classes, pred[0]))

1/1 [==============================] - 0s 117ms/step


{'akiec': -19.072325,
 'bcc': -18.326487,
 'bkl': -8.601684,
 'df': -34.96761,
 'mel': 7.388905,
 'nv': 2.8025546,
 'vasc': -5.3949637}

https://dx.doi.org/10.2139/ssrn.3574612

Melanoma (MEL), Melanomic Neves (NV), Basal Cell Carcinoma (BCC),Actinic Keratosis (AKIEC), Benign Keratosis (BKL), Dermatofibroma (DF), Vascular Lesion (VASC)

In [ ]:
config = model.get_config()
config

In [49]:
import json
json.dumps(config)

'{"name": "model", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 150, 150, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_2"}, "name": "input_2", "inbound_nodes": []}, {"class_name": "Functional", "config": {"name": "xception", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 150, 150, 3], "dtype": "float32", "sparse": false, "ragged": false, "name": "input_1"}, "name": "input_1", "inbound_nodes": []}, {"class_name": "Conv2D", "config": {"name": "block1_conv1", "trainable": false, "dtype": "float32", "filters": 32, "kernel_size": [3, 3], "strides": [2, 2], "padding": "valid", "data_format": "channels_last", "dilation_rate": [1, 1], "groups": 1, "activation": "linear", "use_bias": false, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer

In [ ]:
! pip install gradio
import gradio as gr

In [104]:
def classify_image(inp):
  img = load_img(inp, target_size=(150, 150))
  x = np.array(img)
  X = np.array([x])
  X = preprocess_input(X)
  pred = model.predict(X).flatten()
  # return {classes[i]: float(pred[i]) for i in range(7)}
  # return {'cat': 0.3, 'dog': 0.7}

"""gr.Interface(
    fn=classify_image, 
    inputs=gr.Image(),
    outputs=gr.outputs.Label(num_top_classes=4),
    examples=["320px-Melanoma.jpg"],
).launch()"""

def image_classifier(inp):
    x = np.array(img)
    X = np.array([x])
    X = preprocess_input(X)
    pred = model.predict(X).flatten()
    # return {classes[i]: float([0.3,0.4,0.6,0.2,0.9,0.1,0.5][i]) for i in range(7)}
    return {classes[i]: float(pred[i]) for i in range(7)}

demo = gr.Interface(fn=image_classifier, inputs="image", outputs="label", examples=["320px-Melanoma.jpg"],)
demo.launch()

# classify_image("320px-Melanoma.jpg")

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>